In [1]:
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic

load_dotenv()

KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = "claude-3-opus-20240229"
chat = ChatAnthropic(temperature=0, api_key=KEY, model_name=MODEL)

# response = chat.invoke("Can you tell me a joke?")

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# chain = chat | parser
# chain.invoke("Can you tell me a joke?")

In [5]:
from langchain_community.document_loaders import  PyPDFLoader

loader = PyPDFLoader("Zarchiver.pdf")
pages = loader.load_and_split()


In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)


In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")


In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
  pages, 
  embedding=embeddings
)

In [14]:
retriver = vectorstore.as_retriever()
# retriver.invoke("Zarchiver")

In [13]:
from operator import itemgetter

chain = (
  {
    "context": itemgetter("question") | retriver,
    "question": itemgetter("question")
  }
  | prompt
  | chat
  | parser
)

chain.invoke({"question": "What is Zarchiver"})

'Based on the given context, Zarchiver is an Android application designed for efficient archive file management. It allows users to compress large folders into more manageable sizes and extract the compressed files to access the content when needed.\n\nSome key features of Zarchiver include:\n\n1. Protection - It secures archive folders by applying passwords.\n\n2. Compression and Decompression - It can compress large files into small archive files to save storage space and extract compressed files to access their contents. \n\n3. File management - It allows file operations like copying, moving, deleting, searching files within the app.\n\n4. Supports various archive types like 7zip, zip, bzip2 and more.\n\n5. User-friendly interface with a simple layout that makes it easy to use.\n\nOverall, Zarchiver fills the gap by offering functionality similar to desktop applications like WinRAR, 7-Zip directly on Android devices for managing archive files.'